In [1]:
pip install delta-spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 963.9 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
import random 
from random import randint
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from delta.tables import DeltaTable

In [2]:
spark_conf = (
    SparkConf()
    .set("spark.jars.packages", 'org.apache.hadoop:hadoop-client:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.0.0')
  
    .set("spark.driver.memory", "6g")

    
    .set("spark.hadoop.fs.s3a.endpoint", "minio:9000")
    .set("spark.hadoop.fs.s3a.access.key", "lhN570D4HT3OyEVXgif4")
    .set("spark.hadoop.fs.s3a.secret.key", "VVkigfOUyMLSiq3a8mqFrjqA79rga0cMoAZHZgTA" )
    .set("spark.hadoop.fs.s3a.path.style.access", "true") 
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

sc = SparkContext.getOrCreate(spark_conf)
spark = SparkSession(sc)

# Generate sample data
def generate_sales_data(day, num_rows=100):
    return spark.range(1, num_rows + 1).withColumn("user", F.concat(F.lit("user_"), (F.rand() * 1000).cast("int"))) \
        .withColumn("product", F.concat(F.lit("product_"), (F.rand() * 100).cast("int"))) \
        .withColumn("time", F.lit(day))

#def generate_products_data():
#    return spark.range(1, 11).withColumn("name", F.concat(F.lit("product_"), F.col("id"))) \
#        .withColumn("name", F.concat(F.lit("Product "), F.col("id"))) \
#        .withColumn("price", (F.rand() * 100).cast("double"))

def generate_products_data():
    return spark.range(1, 11).withColumn("name", F.concat(F.lit("product_"), F.col("id"))) \
        .withColumn("price", (F.rand() * 100).cast("double"))

def generate_users_data():
    return spark.range(1, 11).withColumn("user", F.concat(F.lit("user_"), F.col("id"))) \
        .withColumn("name", F.concat(F.lit("User "), F.col("id"))) \
        .withColumn("location", F.concat(F.lit("Location_"), (F.rand() * 10).cast("int")))

In [3]:
# Generate sample data
def generate_sales_data(day, num_rows=100):
    return spark.range(1, num_rows + 1).withColumn("user", F.concat(F.lit("user_"), (F.rand() * 1000).cast("int"))) \
        .withColumn("product", F.concat(F.lit("product_"), (F.rand() * 100).cast("int"))) \
        .withColumn("time", F.lit(day))

def generate_products_data():
    return spark.range(1, 301).withColumn("name", F.concat(F.lit("product_"), F.col("id"))) \
        .withColumn("price", (F.rand() * 100).cast("double"))

def generate_users_data():
    return spark.range(1, 1001).withColumn("user", F.concat(F.lit("user_"), F.col("id"))) \
        .withColumn("name", F.concat(F.lit("User "), F.col("id"))) \
        .withColumn("location", F.concat(F.lit("Location_"), (F.rand() * 10).cast("int")))

sales_data = generate_sales_data(1, 100)
products_data = generate_products_data()
users_data = generate_users_data()

In [4]:
# Create Bronze tables
num_rows = 100
for day in range(1, 4):
    sales_data = generate_sales_data(day, num_rows)
    sales_table_name = f"sale_day_{day}"
    sales_data.write.format("delta").mode("overwrite").save('s3a://assignment3/sales')

In [5]:
products_data = generate_products_data()
products_data.write.format("delta").mode("overwrite").save('s3a://assignment3/products')

In [6]:
users_data = generate_users_data()
users_data.write.format("delta").mode("overwrite").save('s3a://assignment3/users')

In [7]:
products_data.show(5)

+---+---------+------------------+
| id|     name|             price|
+---+---------+------------------+
|  1|product_1| 52.80759514435586|
|  2|product_2|11.414483782059238|
|  3|product_3|18.435399641015003|
|  4|product_4|4.4127595495788485|
|  5|product_5| 67.65239787641933|
+---+---------+------------------+
only showing top 5 rows



In [8]:
users_data.show(5)

+---+------+------+----------+
| id|  user|  name|  location|
+---+------+------+----------+
|  1|user_1|User 1|Location_5|
|  2|user_2|User 2|Location_0|
|  3|user_3|User 3|Location_7|
|  4|user_4|User 4|Location_5|
|  5|user_5|User 5|Location_3|
+---+------+------+----------+
only showing top 5 rows



In [9]:
sales_data.show(5)

+---+--------+----------+----+
| id|    user|   product|time|
+---+--------+----------+----+
|  1|user_992|product_11|   3|
|  2| user_84|product_44|   3|
|  3|user_760|product_38|   3|
|  4|user_461|product_94|   3|
|  5|user_711|product_18|   3|
+---+--------+----------+----+
only showing top 5 rows



In [10]:
sales_data = spark.read.format("delta").load("s3a://assignment3/sales")
products_data = spark.read.format("delta").load("s3a://assignment3/products")
users_data = spark.read.format("delta").load("s3a://assignment3/users")

In [13]:
# Create Silver table
detailed_sales_data = (
    sales_data
    .join(users_data, sales_data["user"] == users_data["user"], "left_outer")
    .join(products_data, sales_data["product"] == products_data["name"], "left_outer")
    .select(
        sales_data["id"].alias("sale_id"),
        sales_data["user"],
        users_data["name"].alias("user_name"),
        users_data["location"],
        sales_data["product"],
        products_data["name"].alias("product_name"),
        products_data["price"],
        sales_data["time"]
    )
)


In [14]:
detailed_sales_data.show(5)

+-------+--------+---------+----------+----------+------------+------------------+----+
|sale_id|    user|user_name|  location|   product|product_name|             price|time|
+-------+--------+---------+----------+----------+------------+------------------+----+
|      1|user_992| User 992|Location_8|product_11|  product_11| 46.18329598989321|   3|
|      2| user_84|  User 84|Location_7|product_44|  product_44| 86.06372809598145|   3|
|      3|user_760| User 760|Location_1|product_38|  product_38|47.167412702636334|   3|
|      4|user_461| User 461|Location_3|product_94|  product_94| 37.35588144771739|   3|
|      5|user_711| User 711|Location_8|product_18|  product_18| 38.37895620958446|   3|
+-------+--------+---------+----------+----------+------------+------------------+----+
only showing top 5 rows



In [16]:
detailed_sales_data.write.format("delta").mode("overwrite").save('s3a://assignment3/silver_data')

In [18]:
detailed_sales_data = spark.read.format("delta").load("s3a://assignment3/silver_data")

In [19]:
# Sum of money per location
money_per_location = (
    detailed_sales_data
    .groupBy("location")
    .agg(F.sum("price").alias("total_money"))
)

In [21]:
money_per_location.show(5)

+----------+------------------+
|  location|       total_money|
+----------+------------------+
|Location_5|455.90583640898194|
|Location_8| 364.3365932116258|
|Location_9| 852.2728701511855|
|Location_4|345.62185527984883|
|Location_3| 939.3026096200543|
+----------+------------------+
only showing top 5 rows



In [20]:
# Amount of sales per product
sales_per_product = (
    detailed_sales_data
    .groupBy("product_name")
    .agg(F.count("sale_id").alias("sales_count"))
)

In [22]:
sales_per_product.show(5)

+------------+-----------+
|product_name|sales_count|
+------------+-----------+
|  product_55|          3|
|  product_98|          1|
|  product_60|          1|
|  product_78|          4|
|  product_53|          4|
+------------+-----------+
only showing top 5 rows



In [23]:
money_per_location.write.format("delta").mode("overwrite").save('s3a://assignment3/money_per_location_golden')

In [ ]:
sales_per_product.write.format("delta").mode("overwrite").save('s3a://assignment3/sales_per_product_golden')